## Pre-trained model usage

### Importing an existing deep learning model for image classification

In [47]:
# The model for the skin cancer classifier

# Importing the libraries
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout
#from tensorflow.keras.layers.core import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [48]:
# Checking if GPU is available
# Tensorflow is installed without CUDA support
import tensorflow as tf
tf.test.is_built_with_cuda()

False

In [49]:
# The paths for the training and validation images
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'

In [50]:
# Declaring a few useful values
num_train_samples = 9013
num_val_samples = 1002
train_batch_size = 10
val_batch_size = 10
image_size = 224

In [51]:
# Declaring how many steps are needed in an iteration
train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [52]:
# Setting up generators
train_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path,
    target_size=(image_size, image_size),
    batch_size=train_batch_size)

valid_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size)

test_batches = ImageDataGenerator(
    preprocessing_function= \
        keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path,
    target_size=(image_size, image_size),
    batch_size=val_batch_size,
    shuffle=False)

Found 38704 images belonging to 7 classes.
Found 1002 images belonging to 7 classes.
Found 1002 images belonging to 7 classes.


In [53]:
# Initialising a MobileNet pre-trained model from Keras and passing it to model object
mobile = keras.applications.mobilenet.MobileNet()

In [54]:
# Displaying a summary of the layers in the model
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

## Transfer learning

### Modifying MobileNet model architecture to retrain it on the data

The **MobileNet pre-trained model** has millions of connections that has already been trained on a large data set. It is reliable at identifying different features hence there is **only a need to retrain a part of it** by 'freezing' its first few layers. **After adding a fully connected layer and through only training the last few layers, a 'new' model is obtained** that can effectively identify basic features and make predictions on relevant new data.

In [55]:
# Modifying the model
# Excluding the last 5 layers of the model
x = mobile.layers[-6].output
# Adding a dropout and dense layer for predictions
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

In [56]:
# Creating a new model with the new outputs
model = Model(inputs=mobile.input, outputs=predictions)

In [57]:
# Seeing a summary of the new layers in the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [58]:
# Freezing the weights of the layers that we aren't training (training the last 23 to avoid memory problem).
#from tensorflow.keras.models import Model
#from tensorflow.keras import Model
for layer in model.layers[:-23]:
    layer.trainable = False

### Training the new model

In [59]:
# Training the model
# Defining Top2 and Top3 Accuracy
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_2_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

In [60]:
# Compiling the model
model.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=[categorical_accuracy, top_2_accuracy, top_3_accuracy])

# Adding weights to make the model more sensitive to melanoma, which is a type of skin cancer
class_weights={
    0: 1.0,  # akiec
    1: 1.0,  # bcc
    2: 1.0,  # bkl
    3: 1.0,  # df
    4: 3.0,  # mel
    5: 1.0,  # nv
    6: 1.0,  # vasc
}

In [61]:
# Declaring the filepath for the saved model
filepath = "model.h5"

In [62]:
# Declaring a checkpoint to save the best version of the model
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1,
                             save_best_only=True, mode='max')

In [63]:
# Reducing the learning rate as the learning stagnates
reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2,
                              verbose=1, mode='max', min_lr=0.00001)

callbacks_list = [checkpoint, reduce_lr]

In [66]:
# Fitting the model
history = model.fit_generator(train_batches,
                              steps_per_epoch=train_steps,
                              class_weight=class_weights,
                              validation_data=valid_batches,
                              validation_steps=val_steps,
                              epochs=5,
                              verbose=1,
                              callbacks=callbacks_list)

Epoch 1/5
901/902 [============================>.] - ETA: 38s - loss: 1.2508 - categorical_accuracy: 0.6164 - top_2_accuracy: 0.8186 - top_3_accuracy: 0.9183 
Epoch 00001: val_top_3_accuracy improved from 0.83034 to 0.91218, saving model to model.h5


OSError: Can't write data (file write failed: time = Fri May 20 08:14:03 2022
, filename = 'model.h5', file descriptor = 81, errno = 28, error message = 'No space left on device', buf = 0x7fd0bf800000, total write size = 1048576, bytes this sub-write = 1048576, bytes actually written = 18446744073709551615, offset = 0)